Plotting the required data
------------------------------------

In this notebook I initialize the connection to the database, run the query to select the required data, and save a figure.

First, I start by initializing the same environment as the fastapi

In [1]:
import json
from datetime import date
from typing import List

from fastapi import FastAPI, File, Query, UploadFile
from fastapi.middleware.cors import CORSMiddleware

from ccfatigue import analyzer
from ccfatigue import dashboarder
from ccfatigue.model import (
    Dashboard, Experience, Plot, SnCurveMethod, SnCurveResult, Test)
from ccfatigue.config import settings
from ccfatigue.services.database import Base, database, engine
from ccfatigue.models.database import *

Base.metadata.create_all(engine)

DB already exists
2022-02-03 18:51:36,456 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-02-03 18:51:36,457 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-03 18:51:36,460 INFO sqlalchemy.engine.Engine select current_schema()
2022-02-03 18:51:36,463 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-03 18:51:36,466 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-02-03 18:51:36,467 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-03 18:51:36,470 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-03 18:51:36,472 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-02-03 18:51:36,474 INFO sqlalchemy.engine.Engine [generated in 0.00193s] {'name': 'experience'}
2022-02-03 18:51:36,481 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.

Here, I wanted to define some of the variable and some initial ORM queries to check that everything is in order.

In [2]:
fatigue = Base.metadata.tables['fatigue_data']

In [3]:
from sqlalchemy.orm import sessionmaker, aliased
from sqlalchemy import func


In [4]:
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()


In [5]:
q1 = session.query(
    fatigue.columns['Folder'], func.max(fatigue.columns['Machine_Load'])).group_by(
    fatigue.columns['Folder']).all()
print(q1)

2022-02-03 18:51:36,617 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-03 18:51:36,621 INFO sqlalchemy.engine.Engine SELECT fatigue_data."Folder" AS "fatigue_data_Folder", max(fatigue_data."Machine_Load") AS max_1 
FROM fatigue_data GROUP BY fatigue_data."Folder"
2022-02-03 18:51:36,623 INFO sqlalchemy.engine.Engine [generated in 0.00111s] {}
[('/home/dt/git/epfl/IT4R-dataeng-hmw/Data/TST_FakeResearcher_2019-09_QS', 5.733086426), ('/home/dt/git/epfl/IT4R-dataeng-hmw/Data/TST_FakeResearcher_2020-06_FA', 35.077697266), ('/home/dt/git/epfl/IT4R-dataeng-hmw/Data/TST_FakeResearcher_2021-06_FA', 31.0), ('/home/dt/git/epfl/IT4R-dataeng-hmw/Data/TST_Khalooei_2021-10_FA', 7.632850098)]


In [6]:
tests = Base.metadata.tables['tests']
q2 = session.query(
    tests.columns["Folder"], tests.columns["Stress Ratio"]).all()
print(q2)

2022-02-03 18:51:37,169 INFO sqlalchemy.engine.Engine SELECT tests."Folder" AS "tests_Folder", tests."Stress Ratio" AS "tests_Stress Ratio" 
FROM tests
2022-02-03 18:51:37,175 INFO sqlalchemy.engine.Engine [generated in 0.00681s] {}
[('/home/dt/git/epfl/IT4R-dataeng-hmw/Data/TST_FakeResearcher_2020-06_FA', 0.1), ('/home/dt/git/epfl/IT4R-dataeng-hmw/Data/TST_FakeResearcher_2020-06_FA', 0.2), ('/home/dt/git/epfl/IT4R-dataeng-hmw/Data/TST_FakeResearcher_2020-06_FA', 0.2), ('/home/dt/git/epfl/IT4R-dataeng-hmw/Data/TST_FakeResearcher_2020-06_FA', 0.2), ('/home/dt/git/epfl/IT4R-dataeng-hmw/Data/TST_FakeResearcher_2020-06_FA', 0.1), ('/home/dt/git/epfl/IT4R-dataeng-hmw/Data/TST_FakeResearcher_2019-09_QS', 0.9), ('/home/dt/git/epfl/IT4R-dataeng-hmw/Data/TST_FakeResearcher_2019-09_QS', 0.9), ('/home/dt/git/epfl/IT4R-dataeng-hmw/Data/TST_FakeResearcher_2019-09_QS', 0.9), ('/home/dt/git/epfl/IT4R-dataeng-hmw/Data/TST_FakeResearcher_2019-09_QS', 0.9), ('/home/dt/git/epfl/IT4R-dataeng-hmw/Data/TST_

Here is the query that I want to run, tested on PSQL, that I wanted to convert to something more legible.

The query I want to run is a bit complex, so I write it down.
First I aggregate to find the maxload, then I join on tests to have the final result

``` sql
SELECT aa.maxload, t."Stress Ratio" 
FROM 
(
  SELECT 
  MAX("Machine_Load") as maxload,  f."Folder", f."TestMetadata_id"
  FROM fatigue_data as f 
  JOIN 
  tests as t 
  ON 
  ( f."Folder" = t."Folder" AND f."TestMetadata_id" = t."Specimen number") 
  GROUP BY 
  f."Folder", f."TestMetadata_id"
) as aa 
JOIN
tests as t 
ON
( aa."Folder" = t."Folder" AND aa."TestMetadata_id" = t."Specimen number")
```

Unfortunately I run of time to convert it as I wanted!

In [7]:
mmax = aliased(func.max(fatigue.columns['Machine_Load']))
ff = fatigue.columns["Folder"]
cf = tests.columns["Folder"]
fid = fatigue.columns["TestMetadata_id"]
tid = tests.columns["Specimen number"]

I wanted to write down an ORM with the request but I got mixed up in the construction of the subqueries...
apologies for the raw execution.

In [8]:
sqlstatement = """SELECT aa.maxload, t."Stress Ratio" FROM ( SELECT MAX("Machine_Load") as maxload,  f."Folder", f."TestMetadata_id"
  FROM fatigue_data as f 
  JOIN 
  tests as t 
  ON 
  ( f."Folder" = t."Folder" AND f."TestMetadata_id" = t."Specimen number") 
  GROUP BY 
  f."Folder", f."TestMetadata_id"
) as aa 
JOIN
tests as t 
ON
( aa."Folder" = t."Folder" AND aa."TestMetadata_id" = t."Specimen number")"""
result = session.execute(sqlstatement).all()

2022-02-03 18:51:37,216 INFO sqlalchemy.engine.Engine SELECT aa.maxload, t."Stress Ratio" FROM ( SELECT MAX("Machine_Load") as maxload,  f."Folder", f."TestMetadata_id"
  FROM fatigue_data as f 
  JOIN 
  tests as t 
  ON 
  ( f."Folder" = t."Folder" AND f."TestMetadata_id" = t."Specimen number") 
  GROUP BY 
  f."Folder", f."TestMetadata_id"
) as aa 
JOIN
tests as t 
ON
( aa."Folder" = t."Folder" AND aa."TestMetadata_id" = t."Specimen number")
2022-02-03 18:51:37,218 INFO sqlalchemy.engine.Engine [generated in 0.00177s] {}


Here I collect the result of the query, and save the coordinates. It's not immediatel clear what x and y are in here, but it should be easy once the query is analyzed.

In [9]:
result
x, y  = ([i[0] for i in result], [i[1] for i in result])


Here, a basic Bokeh plot to show the relation between the two variables.

In [10]:
# importing the modules
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool, BoxSelectTool

output_file("stress_load.html")
TOOLS = [BoxSelectTool(), HoverTool()]

   
# instantiating the figure object
graph = figure(
    title = "Relation between maximum load and maximum stress",
    tools=TOOLS)
graph.circle(x=x, y=y,fill_alpha=0.2, size=10)

   
# displaying the model
show(graph)

Opening in existing browser session.


[156475:156475:0100/000000.445973:ERROR:sandbox_linux.cc(378)] InitializeSandbox() called with multiple threads in process gpu-process.
